In [1]:
cdcurve = {
    "node": ["1D", "3M", "1Y", "2Y", "3Y", "4Y", "5Y"],
    "intrate": [3.55, 3.48, 3.22, 3.00, 2.905, 2.8625, 2.845]
}

In [2]:
import pandas as pd

# 딕셔너리 형태의 cdcurve 데이터
cdcurve = {
    "node": ["1D", "3M", "1Y", "2Y", "3Y", "4Y", "5Y"],
    "intrate": [3.55, 3.48, 3.22, 3.00, 2.905, 2.8625, 2.845]
}
# 데이터프레임으로 데이터 변환 후 basedt 변수 생성(2024-08-02)
df = pd.DataFrame(cdcurve)
basedt = "2024-08-02"
# basedt 컬럼 추가 및 컬럼 재배열
df["basedt"] = basedt
df = df[["basedt", "node", "intrate"]]

금리 커브의 노드 일자

In [3]:
# pandas(데이터 라이브러리), datetime(날짜 라이브러리), re(정규식 라이브러리) import 
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re

# node에서 숫자만 뽑아내는 함수
def extract_numbers(input_string):
	# 숫자인 것 전부 뽑아내서 numbers에 저장
    numbers = re.findall(r'\d+', input_string)
    # "" 공백 없이 합치기
    return ''.join(numbers)    
    
# modified following 만족하는 날짜 return 하는 함수
def get_modf_eom_date(date, holidays):
    # 입수된 날짜의 월 변수에 저장
    origin_month = date.month
    # 입수된 날짜의 값이 5(토), 6(일), 휴일 일 경우 하루씩 더하는 작업 -> 휴일일 경우 
    # 다음 영업일로 변환하는 loop 문
    while date.weekday() >= 5 or date in holidays:
        date += timedelta(days=1)
    # 다음영업일로 변환하는 과정에서 월이 바뀌었을 경우
    if date.month != origin_month:
    	# 다음월의 1일에서 직전일로 end_of_month 세팅(직전월의 마지막 날짜)
        end_of_month = datetime(date.year, origin_month + 1, 1) - timedelta(days=1)
        # 직전월의 마지막 날자가 휴일일 경우(5(토), 6(일), 휴일) 휴일이 아닌 날짜가 될 때까지
        # 직전일로 변경하는 loop 문
        while end_of_month.weekday() >= 5 or end_of_month in holidays:
            end_of_month -= timedelta(days=1)
        date = end_of_month
    return date

# node의 조정된 날짜를 구하는 함수
def get_adj_date(inputdate:datetime.date, node:str, holidays):
    # node 값이 D로 끝날 경우
    if node[-1] == "D":
    	# extract_numbers 함수로 며칠 뒤의 데이터인지 값 추출
        interval_number = extract_numbers(node)
        interval = int(interval_number) # 숫자로 형식 변환
        date = inputdate + relativedelta(days=interval) # 날짜 더하기
        # date를 modified following 을 만족하는 날짜로 변경
        adjdate = get_modf_eom_date(date, holidays).strftime("%Y-%m-%d")
    # node 값이 M으로 끝날 경우
    elif node[-1] == "M":
   	    # extract_numbers 함수로 몇 달 뒤의 데이터인지 값 추출
        interval_number = extract_numbers(node)
        interval = int(interval_number) # 숫자로 형식 변환
        date = inputdate + relativedelta(months=interval) # 월 수 더하기
        # date를 modified following 을 만족하는 날짜로 변경
        adjdate = get_modf_eom_date(date, holidays).strftime("%Y-%m-%d")
    # node 값이 Y로 끝날 경우
    elif node[-1] == "Y":
    	# extract_numbers 함수로 몇 년 뒤의 데이터인지 값 추출
        interval_number = extract_numbers(node)
        interval = int(interval_number) # 숫자로 형식 변환
        date = inputdate + relativedelta(years=interval) # 년 수 더하기
        # date를 modified following 을 만족하는 날짜로 변경
        adjdate = get_modf_eom_date(date, holidays).strftime("%Y-%m-%d")
    # 예외처리
    else:
        adjdate = inputdate
    return adjdate

# 'basedt", "node", "intrate"으로 구성된 dataframe을 받아서 "nodedate"을 추가하고 
# get_adj_date을 활용해서 "nodedate"컬럼을 값을 채워나가는 함수
def get_node_date(df, basedt, holidays):
	# 빈 "nodedate" 컬럼 추가
    df["nodedate"] = ""
    # 입수된 basedt를 날짜 형식으로 변경
    basedt = datetime.strptime(basedt, "%Y-%m-%d")
    # 입수된 dataframe 의 node 별로 get_adj_date 함수로 작업해서 "nodedate"컬럼에 값 채우기
    for i in df["node"]:
        adjdate = get_adj_date(basedt, i, holidays)
        df.loc[df["node"]==i, "nodedate"] = adjdate
    return df[["basedt", "node", "nodedate", "intrate"]]

KRX에서는 주기를 3개월 가정하기에 단위로 이자율 계산해서 bootstrapping통해 무이표금리
선형보간법으로 비기준만기 스왑금리 커브 만듦 (금리와 금리사이 값)

구간일수 구하기

In [4]:
# 라이브러리 사용
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import re

# modified following 과 end of month 규칙이 적용된 날짜를 구하는 함수
def get_modf_eom_date(date, holidays):
    origin_month = date.month
    while date.weekday() >= 5 or date in holidays:
        date += timedelta(days=1)
    if date.month != origin_month:
        end_of_month = datetime(date.year, origin_month + 1, 1) - timedelta(days=1)
        while end_of_month.weekday() >= 5 or end_of_month in holidays:
            end_of_month -= timedelta(days=1)
        date = end_of_month
    return date

# Actual 365 날짜 규칙을 바탕으로 시작일과 종료일의 날짜를 계산하는 함수
def get_timeinterval_a365f(startdt, enddt):
    startdt = datetime.strptime(startdt, '%Y-%m-%d')
    enddt = datetime.strptime(enddt, '%Y-%m-%d')
    holidays = holidays
    holidays = [datetime.strptime(holiday, '%Y-%m-%d') for holiday in holidays]
    startdt = get_modf_eom_date(startdt)
    enddt = get_modf_eom_date(enddt)
    actdays = (enddt - startdt).days
    return actdays

선형보간법 함수

In [5]:
def linear_interpolate(stdate, eddate, strate, edrate, targetdate):
    tottimediff = get_timeinterval_a365f(stdate, eddate)
    timediff = get_timeinterval_a365f(stdate, targetdate)
    intdiff = edrate - strate
    targetrate = strate + timediff / tottimediff * intdiff
    return targetrate

3개월 단위 스왑커브

In [6]:
def config_curve_by_unit(df: pd.DataFrame):
        add_df = []
        basedt = datetime.strftime(basedt, "%Y-%m-%d")
        for i in df.index:
            if df.at[i, "node"] == "3M":
                targetdate1 = get_adj_date(basedt, "6M")
                targetdate2 = get_adj_date(basedt, "9M")
                stdate = df[df["node"] == "3M"]["nodedate"].iloc[0]
                eddate = df[df["node"] == "1Y"]["nodedate"].iloc[0]
                strate = df[df["node"] == "3M"]["intrate"].iloc[0]
                edrate = df[df["node"] == "1Y"]["intrate"].iloc[0]
                add_rate1 = linear_interpolate(stdate, eddate, strate, edrate, targetdate1)
                add_row1 = [basedt, "6M", targetdate1, add_rate1]
                add_df.append(add_row1)
                add_rate2 = linear_interpolate(stdate, eddate, strate, edrate, targetdate2)
                add_row2 = [basedt, "9M", targetdate2, add_rate2]
                add_df.append(add_row2)
            elif df.at[i, "node"][-1] == "Y" and i != df.index[-1]:
                num_of_years = int(extract_numbers(df.at[i, "node"]))
                month1 = str(num_of_years * 12 + 3) + "M"
                month2 = str(num_of_years * 12 + 6) + "M"
                month3 = str(num_of_years * 12 + 9) + "M" 
                targetdate1 = get_adj_date(basedt, month1)
                targetdate2 = get_adj_date(basedt, month2)
                targetdate3 = get_adj_date(basedt, month3)
                stdate = df.at[i, "nodedate"]
                eddate = df.at[i+1, "nodedate"]
                strate = df.at[i, "intrate"]
                edrate = df.at[i+1, "intrate"]
                add_rate1 = linear_interpolate(stdate, eddate, strate, edrate, targetdate1)
                add_row1 = [basedt, month1, targetdate1, add_rate1]
                add_df.append(add_row1)
                add_rate2 = linear_interpolate(stdate, eddate, strate, edrate, targetdate2)
                add_row2 = [basedt, month2, targetdate2, add_rate2]
                add_df.append(add_row2)
                add_rate3 = linear_interpolate(stdate, eddate, strate, edrate, targetdate3)
                add_row3 = [basedt, month3, targetdate3, add_rate3]
                add_df.append(add_row3)
        add_df = pd.DataFrame(add_df, columns=["basedt", "node", "nodedate", "intrate"])
        df = pd.concat([df, add_df])
        df_sorted = df.sort_values(by="nodedate", ascending=True)
        df_sorted_reset = df_sorted.reset_index(drop=True)
        return df_sorted_reset